# Import Libraries & Data

In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_pickle('dolos_openface_merged.pkl')

In [3]:
dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label,video_id
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190800,146,0,5.80,0.93,1,0.220692,-0.050918,-0.974014,0.050902,-0.056871,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190801,147,0,5.84,0.98,1,0.233077,-0.042794,-0.971516,0.053544,-0.046702,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190802,148,0,5.88,0.98,1,0.235445,-0.043778,-0.970901,0.054714,-0.047773,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190803,149,0,5.92,0.98,1,0.236376,-0.041068,-0.970793,0.054574,-0.044504,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16


# Aggregate all data again

In [4]:
# at first we used only the AU column -> no good results so we added features for pose and gaze as well
#dataset.columns.values

In [5]:
dataset.sort_values(['video_id', 'face_id', 'timestamp'], inplace = True)

In [13]:
# keep only au, pose & gaze columns

au_cols = [c for c in dataset.columns if c.endswith("_r")]

pose_cols = [c for c in dataset.columns if c.startswith("pose_")]

gaze_cols = [c for c in dataset.columns if c.startswith("gaze_")]

final_columns = au_cols + pose_cols + gaze_cols
#final_columns

## Compute aggregates (mean/max/std) per video

In [15]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean())
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max())
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std())
video_std

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,pose_Ry,pose_Rz,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.131243,0.085327,0.562402,0.033261,0.132124,0.181448,0.057571,0.446373,0.352630,0.328930,...,0.498067,0.071552,0.350469,0.050873,0.060047,0.324428,0.044437,0.103153,0.375321,0.044396
AN_WILTY_EP15_lie11,0.314456,0.219906,0.000000,0.130074,0.005639,0.000000,0.095572,0.297845,0.164786,0.574241,...,0.058292,0.065948,0.036312,0.052150,0.009700,0.044876,0.045695,0.008450,0.040575,0.050515
AN_WILTY_EP15_lie12,0.095647,0.070883,0.217000,0.079565,0.381883,0.461531,0.253215,0.538801,0.348212,0.470164,...,0.036717,0.035931,0.020637,0.028700,0.011721,0.023971,0.026333,0.007802,0.024190,0.029536
AN_WILTY_EP15_lie13,0.095652,0.076767,0.000000,0.048791,0.109656,0.129917,0.066048,0.419778,0.430428,0.564710,...,0.050602,0.050558,0.028064,0.018287,0.010192,0.033853,0.019273,0.005695,0.032295,0.019413
AN_WILTY_EP15_lie14,0.385574,0.348092,0.000000,0.113538,0.440128,0.313956,0.082497,0.566972,0.704785,0.606506,...,0.172690,0.041356,0.101068,0.041543,0.024627,0.098393,0.043941,0.009518,0.103674,0.044448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.147764,0.145458,0.101270,0.152342,0.292310,0.311208,0.064355,0.373886,0.475253,0.197343,...,0.063470,0.061649,0.044414,0.049891,0.010820,0.038345,0.038554,0.002456,0.042085,0.044785
YW_WILTY_EP70_truth6,0.527518,0.200851,0.521999,0.367288,0.429061,0.612170,0.167712,0.636275,0.572024,0.434525,...,0.408428,0.178965,0.279648,0.118295,0.104138,0.239958,0.109123,0.117627,0.301140,0.153910
YW_WILTY_EP70_truth7,0.436879,0.281196,0.361892,0.057731,0.114382,0.298365,0.099114,0.140731,0.228547,0.000000,...,0.126236,0.032992,0.098324,0.046222,0.028750,0.089800,0.047720,0.039305,0.103208,0.050876


In [16]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean().add_prefix("stau_mean_"))
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max().add_prefix("stau_max_"))
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std().add_prefix("stau_std_"))
video_std

,stau_std_AU01_r,stau_std_AU02_r,stau_std_AU04_r,stau_std_AU05_r,stau_std_AU06_r,stau_std_AU07_r,stau_std_AU09_r,stau_std_AU10_r,stau_std_AU12_r,stau_std_AU14_r,...,stau_std_pose_Ry,stau_std_pose_Rz,stau_std_gaze_0_x,stau_std_gaze_0_y,stau_std_gaze_0_z,stau_std_gaze_1_x,stau_std_gaze_1_y,stau_std_gaze_1_z,stau_std_gaze_angle_x,stau_std_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.131243,0.085327,0.562402,0.033261,0.132124,0.181448,0.057571,0.446373,0.352630,0.328930,...,0.498067,0.071552,0.350469,0.050873,0.060047,0.324428,0.044437,0.103153,0.375321,0.044396
AN_WILTY_EP15_lie11,0.314456,0.219906,0.000000,0.130074,0.005639,0.000000,0.095572,0.297845,0.164786,0.574241,...,0.058292,0.065948,0.036312,0.052150,0.009700,0.044876,0.045695,0.008450,0.040575,0.050515
AN_WILTY_EP15_lie12,0.095647,0.070883,0.217000,0.079565,0.381883,0.461531,0.253215,0.538801,0.348212,0.470164,...,0.036717,0.035931,0.020637,0.028700,0.011721,0.023971,0.026333,0.007802,0.024190,0.029536
AN_WILTY_EP15_lie13,0.095652,0.076767,0.000000,0.048791,0.109656,0.129917,0.066048,0.419778,0.430428,0.564710,...,0.050602,0.050558,0.028064,0.018287,0.010192,0.033853,0.019273,0.005695,0.032295,0.019413
AN_WILTY_EP15_lie14,0.385574,0.348092,0.000000,0.113538,0.440128,0.313956,0.082497,0.566972,0.704785,0.606506,...,0.172690,0.041356,0.101068,0.041543,0.024627,0.098393,0.043941,0.009518,0.103674,0.044448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.147764,0.145458,0.101270,0.152342,0.292310,0.311208,0.064355,0.373886,0.475253,0.197343,...,0.063470,0.061649,0.044414,0.049891,0.010820,0.038345,0.038554,0.002456,0.042085,0.044785
YW_WILTY_EP70_truth6,0.527518,0.200851,0.521999,0.367288,0.429061,0.612170,0.167712,0.636275,0.572024,0.434525,...,0.408428,0.178965,0.279648,0.118295,0.104138,0.239958,0.109123,0.117627,0.301140,0.153910
YW_WILTY_EP70_truth7,0.436879,0.281196,0.361892,0.057731,0.114382,0.298365,0.099114,0.140731,0.228547,0.000000,...,0.126236,0.032992,0.098324,0.046222,0.028750,0.089800,0.047720,0.039305,0.103208,0.050876


In [25]:
agg_1 = video_mean.merge(video_max, on = 'video_id')
agg_1

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,stau_max_pose_Ry,stau_max_pose_Rz,stau_max_gaze_0_x,stau_max_gaze_0_y,stau_max_gaze_0_z,stau_max_gaze_1_x,stau_max_gaze_1_y,stau_max_gaze_1_z,stau_max_gaze_angle_x,stau_max_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.106800,0.047600,0.199600,0.016667,0.069733,0.106400,0.033333,0.731200,1.136267,0.864133,...,1.191,0.194,0.456900,0.323675,-0.673897,0.311285,0.309516,-0.594778,0.404,0.344
AN_WILTY_EP15_lie11,0.205873,0.111587,0.000000,0.059683,0.001905,0.000000,0.048254,0.771746,0.444762,0.679206,...,-0.232,0.071,0.384589,0.298383,-0.891499,0.215192,0.287780,-0.938242,0.312,0.310
AN_WILTY_EP15_lie12,0.070300,0.041700,0.286000,0.038700,0.399800,0.402100,0.107500,1.716800,0.532600,0.783100,...,-0.187,-0.022,0.353507,0.237858,-0.905326,0.182771,0.245497,-0.952012,0.281,0.253
AN_WILTY_EP15_lie13,0.070400,0.040900,0.000000,0.028200,0.053300,0.056200,0.039500,1.030600,0.747300,0.639100,...,-0.125,0.074,0.313348,0.230281,-0.926520,0.152492,0.211950,-0.968918,0.240,0.228
AN_WILTY_EP15_lie14,0.230867,0.154267,0.000000,0.053000,0.658467,0.256333,0.042867,1.299200,2.022067,1.409867,...,0.244,-0.027,0.451158,0.327559,-0.869863,0.296549,0.348091,-0.926410,0.391,0.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.081100,0.069300,0.094800,0.060200,0.388800,0.592700,0.038300,0.189900,0.955500,0.071000,...,-0.006,0.115,0.338564,0.064183,-0.939797,0.144736,0.038746,-0.989263,0.245,0.053
YW_WILTY_EP70_truth6,0.330800,0.089600,0.666960,0.157680,0.593360,0.774000,0.072000,0.671520,0.986080,0.256240,...,1.252,-0.012,0.362173,0.397870,-0.613924,0.175185,0.361579,-0.579364,0.279,0.560
YW_WILTY_EP70_truth7,0.276667,0.135067,0.380400,0.026800,0.197067,0.571200,0.048133,0.023067,0.389867,0.000000,...,0.987,0.057,0.214764,0.162420,-0.746404,0.035922,0.149070,-0.657141,0.126,0.218


In [26]:
agg_2 = agg_1.merge(video_std, on = 'video_id')
agg_2

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,stau_std_pose_Ry,stau_std_pose_Rz,stau_std_gaze_0_x,stau_std_gaze_0_y,stau_std_gaze_0_z,stau_std_gaze_1_x,stau_std_gaze_1_y,stau_std_gaze_1_z,stau_std_gaze_angle_x,stau_std_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.106800,0.047600,0.199600,0.016667,0.069733,0.106400,0.033333,0.731200,1.136267,0.864133,...,0.498067,0.071552,0.350469,0.050873,0.060047,0.324428,0.044437,0.103153,0.375321,0.044396
AN_WILTY_EP15_lie11,0.205873,0.111587,0.000000,0.059683,0.001905,0.000000,0.048254,0.771746,0.444762,0.679206,...,0.058292,0.065948,0.036312,0.052150,0.009700,0.044876,0.045695,0.008450,0.040575,0.050515
AN_WILTY_EP15_lie12,0.070300,0.041700,0.286000,0.038700,0.399800,0.402100,0.107500,1.716800,0.532600,0.783100,...,0.036717,0.035931,0.020637,0.028700,0.011721,0.023971,0.026333,0.007802,0.024190,0.029536
AN_WILTY_EP15_lie13,0.070400,0.040900,0.000000,0.028200,0.053300,0.056200,0.039500,1.030600,0.747300,0.639100,...,0.050602,0.050558,0.028064,0.018287,0.010192,0.033853,0.019273,0.005695,0.032295,0.019413
AN_WILTY_EP15_lie14,0.230867,0.154267,0.000000,0.053000,0.658467,0.256333,0.042867,1.299200,2.022067,1.409867,...,0.172690,0.041356,0.101068,0.041543,0.024627,0.098393,0.043941,0.009518,0.103674,0.044448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.081100,0.069300,0.094800,0.060200,0.388800,0.592700,0.038300,0.189900,0.955500,0.071000,...,0.063470,0.061649,0.044414,0.049891,0.010820,0.038345,0.038554,0.002456,0.042085,0.044785
YW_WILTY_EP70_truth6,0.330800,0.089600,0.666960,0.157680,0.593360,0.774000,0.072000,0.671520,0.986080,0.256240,...,0.408428,0.178965,0.279648,0.118295,0.104138,0.239958,0.109123,0.117627,0.301140,0.153910
YW_WILTY_EP70_truth7,0.276667,0.135067,0.380400,0.026800,0.197067,0.571200,0.048133,0.023067,0.389867,0.000000,...,0.126236,0.032992,0.098324,0.046222,0.028750,0.089800,0.047720,0.039305,0.103208,0.050876


In [27]:
video_dau = (
    dataset
    .groupby("video_id")[final_columns]
    .apply(lambda df: np.nanmean(np.abs(np.diff(df.values, axis=0)), axis=0))
    .apply(pd.Series)
    .set_axis([f"movaver_{c}" for c in final_columns], axis=1)
)

video_dau

,movaver_AU01_r,movaver_AU02_r,movaver_AU04_r,movaver_AU05_r,movaver_AU06_r,movaver_AU07_r,movaver_AU09_r,movaver_AU10_r,movaver_AU12_r,movaver_AU14_r,...,movaver_pose_Ry,movaver_pose_Rz,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.032162,0.023243,0.037973,0.011081,0.020676,0.037568,0.012973,0.115000,0.088919,0.125676,...,0.026649,0.015797,0.018746,0.008842,0.005197,0.017153,0.008852,0.006678,0.019851,0.009919
AN_WILTY_EP15_lie11,0.054355,0.038226,0.000000,0.014032,0.001290,0.000000,0.016452,0.081290,0.054355,0.093710,...,0.007952,0.004984,0.005066,0.006233,0.002557,0.005181,0.006581,0.001783,0.005323,0.006871
AN_WILTY_EP15_lie12,0.026465,0.018586,0.060606,0.019495,0.066061,0.075758,0.029697,0.111414,0.082323,0.142121,...,0.006465,0.004707,0.004262,0.005876,0.001997,0.003907,0.006077,0.001445,0.004182,0.006313
AN_WILTY_EP15_lie13,0.032727,0.022626,0.000000,0.016162,0.011717,0.020606,0.013232,0.098788,0.089596,0.111818,...,0.007576,0.004323,0.004902,0.005377,0.001714,0.004680,0.005578,0.001156,0.004828,0.005646
AN_WILTY_EP15_lie14,0.051544,0.043221,0.000000,0.020940,0.071812,0.061275,0.014497,0.133826,0.112416,0.117584,...,0.012832,0.007054,0.007092,0.007637,0.002924,0.007712,0.007352,0.002233,0.007698,0.008040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.025859,0.021919,0.023636,0.035758,0.052020,0.085960,0.018586,0.038384,0.072525,0.025253,...,0.012313,0.005788,0.008673,0.008245,0.002090,0.007240,0.007427,0.000551,0.008081,0.007960
YW_WILTY_EP70_truth6,0.059758,0.025161,0.074274,0.027500,0.073145,0.100323,0.023710,0.101129,0.080161,0.070161,...,0.029500,0.013097,0.019320,0.014553,0.006026,0.017309,0.014012,0.005595,0.020008,0.016129
YW_WILTY_EP70_truth7,0.063649,0.029189,0.054459,0.010135,0.030676,0.086486,0.019459,0.017027,0.037027,0.000000,...,0.018986,0.005027,0.014628,0.008277,0.004109,0.012646,0.007877,0.004876,0.014703,0.008959


In [28]:
agg_3 = agg_2.merge(video_dau, on = 'video_id')
agg_3

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,movaver_pose_Ry,movaver_pose_Rz,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.106800,0.047600,0.199600,0.016667,0.069733,0.106400,0.033333,0.731200,1.136267,0.864133,...,0.026649,0.015797,0.018746,0.008842,0.005197,0.017153,0.008852,0.006678,0.019851,0.009919
AN_WILTY_EP15_lie11,0.205873,0.111587,0.000000,0.059683,0.001905,0.000000,0.048254,0.771746,0.444762,0.679206,...,0.007952,0.004984,0.005066,0.006233,0.002557,0.005181,0.006581,0.001783,0.005323,0.006871
AN_WILTY_EP15_lie12,0.070300,0.041700,0.286000,0.038700,0.399800,0.402100,0.107500,1.716800,0.532600,0.783100,...,0.006465,0.004707,0.004262,0.005876,0.001997,0.003907,0.006077,0.001445,0.004182,0.006313
AN_WILTY_EP15_lie13,0.070400,0.040900,0.000000,0.028200,0.053300,0.056200,0.039500,1.030600,0.747300,0.639100,...,0.007576,0.004323,0.004902,0.005377,0.001714,0.004680,0.005578,0.001156,0.004828,0.005646
AN_WILTY_EP15_lie14,0.230867,0.154267,0.000000,0.053000,0.658467,0.256333,0.042867,1.299200,2.022067,1.409867,...,0.012832,0.007054,0.007092,0.007637,0.002924,0.007712,0.007352,0.002233,0.007698,0.008040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.081100,0.069300,0.094800,0.060200,0.388800,0.592700,0.038300,0.189900,0.955500,0.071000,...,0.012313,0.005788,0.008673,0.008245,0.002090,0.007240,0.007427,0.000551,0.008081,0.007960
YW_WILTY_EP70_truth6,0.330800,0.089600,0.666960,0.157680,0.593360,0.774000,0.072000,0.671520,0.986080,0.256240,...,0.029500,0.013097,0.019320,0.014553,0.006026,0.017309,0.014012,0.005595,0.020008,0.016129
YW_WILTY_EP70_truth7,0.276667,0.135067,0.380400,0.026800,0.197067,0.571200,0.048133,0.023067,0.389867,0.000000,...,0.018986,0.005027,0.014628,0.008277,0.004109,0.012646,0.007877,0.004876,0.014703,0.008959


In [30]:
data_temp = (dataset[['video_id', 'face_id', 'label']].drop_duplicates(subset = 'video_id'))
data_temp

,video_id,face_id,label
175,AN_WILTY_EP15_lie10,0,lie
262,AN_WILTY_EP15_lie11,0,lie
325,AN_WILTY_EP15_lie12,0,lie
425,AN_WILTY_EP15_lie13,0,lie
525,AN_WILTY_EP15_lie14,0,lie
...,...,...,...
189480,YW_WILTY_EP70_truth5,0,truth
189580,YW_WILTY_EP70_truth6,0,truth
189705,YW_WILTY_EP70_truth7,0,truth
189780,YW_WILTY_EP70_truth8,0,truth


In [31]:
movaver_dataset = agg_3.merge(data_temp, on = 'video_id')
movaver_dataset

,video_id,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,...,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y,face_id,label
0,AN_WILTY_EP15_lie10,0.106800,0.047600,0.199600,0.016667,0.069733,0.106400,0.033333,0.731200,1.136267,...,0.018746,0.008842,0.005197,0.017153,0.008852,0.006678,0.019851,0.009919,0,lie
1,AN_WILTY_EP15_lie11,0.205873,0.111587,0.000000,0.059683,0.001905,0.000000,0.048254,0.771746,0.444762,...,0.005066,0.006233,0.002557,0.005181,0.006581,0.001783,0.005323,0.006871,0,lie
2,AN_WILTY_EP15_lie12,0.070300,0.041700,0.286000,0.038700,0.399800,0.402100,0.107500,1.716800,0.532600,...,0.004262,0.005876,0.001997,0.003907,0.006077,0.001445,0.004182,0.006313,0,lie
3,AN_WILTY_EP15_lie13,0.070400,0.040900,0.000000,0.028200,0.053300,0.056200,0.039500,1.030600,0.747300,...,0.004902,0.005377,0.001714,0.004680,0.005578,0.001156,0.004828,0.005646,0,lie
4,AN_WILTY_EP15_lie14,0.230867,0.154267,0.000000,0.053000,0.658467,0.256333,0.042867,1.299200,2.022067,...,0.007092,0.007637,0.002924,0.007712,0.007352,0.002233,0.007698,0.008040,0,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,YW_WILTY_EP70_truth5,0.081100,0.069300,0.094800,0.060200,0.388800,0.592700,0.038300,0.189900,0.955500,...,0.008673,0.008245,0.002090,0.007240,0.007427,0.000551,0.008081,0.007960,0,truth
1465,YW_WILTY_EP70_truth6,0.330800,0.089600,0.666960,0.157680,0.593360,0.774000,0.072000,0.671520,0.986080,...,0.019320,0.014553,0.006026,0.017309,0.014012,0.005595,0.020008,0.016129,0,truth
1466,YW_WILTY_EP70_truth7,0.276667,0.135067,0.380400,0.026800,0.197067,0.571200,0.048133,0.023067,0.389867,...,0.014628,0.008277,0.004109,0.012646,0.007877,0.004876,0.014703,0.008959,0,truth
1467,YW_WILTY_EP70_truth8,0.186700,0.075200,0.167500,0.287900,0.746200,1.031800,0.048500,0.318800,1.385900,...,0.011338,0.013244,0.002144,0.010528,0.012342,0.001629,0.011131,0.013121,0,truth


In [33]:
movaver_dataset.columns.values

array(['video_id', 'stau_mean_AU01_r', 'stau_mean_AU02_r',
       'stau_mean_AU04_r', 'stau_mean_AU05_r', 'stau_mean_AU06_r',
       'stau_mean_AU07_r', 'stau_mean_AU09_r', 'stau_mean_AU10_r',
       'stau_mean_AU12_r', 'stau_mean_AU14_r', 'stau_mean_AU15_r',
       'stau_mean_AU17_r', 'stau_mean_AU20_r', 'stau_mean_AU23_r',
       'stau_mean_AU25_r', 'stau_mean_AU26_r', 'stau_mean_AU45_r',
       'stau_mean_pose_Tx', 'stau_mean_pose_Ty', 'stau_mean_pose_Tz',
       'stau_mean_pose_Rx', 'stau_mean_pose_Ry', 'stau_mean_pose_Rz',
       'stau_mean_gaze_0_x', 'stau_mean_gaze_0_y', 'stau_mean_gaze_0_z',
       'stau_mean_gaze_1_x', 'stau_mean_gaze_1_y', 'stau_mean_gaze_1_z',
       'stau_mean_gaze_angle_x', 'stau_mean_gaze_angle_y',
       'stau_max_AU01_r', 'stau_max_AU02_r', 'stau_max_AU04_r',
       'stau_max_AU05_r', 'stau_max_AU06_r', 'stau_max_AU07_r',
       'stau_max_AU09_r', 'stau_max_AU10_r', 'stau_max_AU12_r',
       'stau_max_AU14_r', 'stau_max_AU15_r', 'stau_max_AU17_r',
     

## Compute temporal aggregates (mean/max/var) per video

In [ ]:
# # compute aggregates (mean/max/std) per video pt.2

# grouped = dataset.groupby(['video_id', 'face_id', 'label'], sort = False)

# # create temporal diff for AU columns
# for c in au_cols:
#     dataset[f"dau_{c}"] = grouped[c].diff().fillna(0)
#     dataset[f"adau_{c}"] = grouped[f"dau_{c}"].transform(np.abs)

# # create temporal diff for gaze/pose columns
# for c in au_cols:
#     dataset[f"dgp_{c}"] = grouped[c].diff().fillna(0)
#     dataset[f"adgp_{c}"] = grouped[f"dgp_{c}"].transform(np.abs)

# dataset

In [ ]:
#dataset.columns.values

In [ ]:
# # keep only diff_au, adiff_au, diff_gp & adiff_gp columns

# diff_au_cols = [c for c in dataset.columns if c.startswith('dau_')]

# adiff_au_cols = [c for c in dataset.columns if c.startswith('adau_')]

# diff_gp_cols = [c for c in dataset.columns if c.startswith('dgp_')]

# adiff_gp_cols = [c for c in dataset.columns if c.startswith('adgp_')]

# #final_columns = au_cols + pose_cols + pose_cols
# #final_columns

In [ ]:
# # final step: group by & aggregate

# # AU diffs
# dau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].mean().add_prefix("dau_mean_")
# dau_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].min().add_prefix("dau_min_")
# dau_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].max().add_prefix("dau_max_")
# dau_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].var().add_prefix("dau_var_")

# # AU absolute diffs
# adau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].mean().add_prefix("adau_mean_")
# adau_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].min().add_prefix("adau_min_")
# adau_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].max().add_prefix("adau_max_")
# adau_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].var().add_prefix("adau_var_")

# # gaze & pose diffs
# dgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].mean().add_prefix("dgp_mean_")
# dgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].min().add_prefix("dgp_min_")
# dgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].max().add_prefix("dgp_max_")
# dgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].var().add_prefix("dgp_var_")

# # gaze & pose absolute diffs
# adgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].mean().add_prefix("adgp_mean_")
# adgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].min().add_prefix("adgp_min_")
# adgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].max().add_prefix("adgp_max_")
# adgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].var().add_prefix("adgp_var_")

In [ ]:
# # union all aggregates in one table at last

# agg_blocks = [
#     dau_mean, dau_min, dau_max, dau_var, # AU diffs
#     adau_mean, adau_min, adau_max, adau_var,  # AU absolute diffs
#     dgp_mean, dgp_min, dgp_max, dgp_var,   # gaze & pose diffs
#     adgp_mean, adgp_min, adgp_max, adgp_var # gaze & pose absolute diffs
# ]

# for agg in agg_blocks:
#     if agg.empty:
#         continue
#     data_temp = data_temp.merge(agg.reset_index(), on = ['video_id', 'face_id', 'label'], how = 'left')

# Output Files

#### ---> From now on we have one row per video with AU, Pose, Gaze mean/max/std & label

In [ ]:
#data_temp.columns

In [34]:
movaver_dataset.to_pickle('movaver_dataset.pkl')

In [ ]:
#data_temp.to_pickle('dolos_aggr_temporal_v2.pkl')

In [ ]:
# dataset_st_mean.to_pickle('dolos_aggr_mean_v2.pkl')
# dataset_st_max.to_pickle('dolos_aggr_max_v2.pkl')
# dataset_st_std.to_pickle('dolos_aggr_std_v2.pkl')